In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.log import Log
from selenium.webdriver.chrome.options import Options
import time
import json
#import trio

In [2]:

# Set Chrome options
chrome_options = Options()

# Enable browser logging
chrome_options.add_argument("--log-level=ALL")

# Create ChromeDriver instance
driver = webdriver.Chrome(options=chrome_options)

# Load a website
driver.get('https://wyborcza.pl/0,0.html')

# Print messages
for entry in driver.get_log('browser'):
    print(entry)

{'level': 'SEVERE', 'message': "https://wyborcza.pl/0,0.html 1061:56 Uncaught SyntaxError: Unexpected token ';'", 'source': 'javascript', 'timestamp': 1687585048407}
{'level': 'SEVERE', 'message': 'https://wyborcza.pl/0,0.html 1069:4 Uncaught ReferenceError: domain_marketing is not defined', 'source': 'javascript', 'timestamp': 1687585048407}
{'level': 'WARNING', 'message': 'https://bis.gazeta.pl/info/mapa2.js 4 A parser-blocking, cross site (i.e. different eTLD+1) script, https://gazeta.hit.gemius.pl/hmapxy.js, is invoked via document.write. The network request for this script MAY be blocked by the browser in this or a future page load due to poor network connectivity. If blocked in this page load, it will be confirmed in a subsequent console message. See https://www.chromestatus.com/feature/5718547946799104 for more details.', 'source': 'javascript', 'timestamp': 1687585048668}
{'level': 'WARNING', 'message': 'https://bis.gazeta.pl/info/mapa2.js 4 A parser-blocking, cross site (i.e. 

In [32]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Set Chrome options
chrome_options = Options()

# Enable browser logging
chrome_options.add_argument("--enable-logging")
chrome_options.add_argument("--log-level=0")

# Set logging preferences to capture network logs
chrome_prefs = {
    'browser': 'ALL',
    'goog:loggingPrefs': {
        'performance': 'ALL'
    }
}
chrome_options.add_experimental_option('prefs', chrome_prefs)

# Create ChromeDriver instance
driver = webdriver.Chrome(options=chrome_options)

# Load a website
driver.get('https://wyborcza.pl/0,0.html')
start_time = time.time()

# Collect logs for 30 seconds
while time.time() - start_time < 30:
    logs = driver.get_log('browser')
    for log in logs:
        message = log['message']
        print(log['source'],message)
        
        
# while time.time() - start_time < 30:
#     logs = driver.get_log('performance') #nie działa 
#     for log in logs:
#         message = log['message']
#         if 'Network.requestWillBeSent' in message:
#             if 'request' in message:
#                 request = log['message']['params']['request']
#                 if request['method'] == 'POST' or request['method'] == 'GET':
#                     print(message)
        
        
# Close the browser
driver.quit()

javascript https://wyborcza.pl/0,0.html 1061:56 Uncaught SyntaxError: Unexpected token ';'
javascript https://wyborcza.pl/0,0.html 1069:4 Uncaught ReferenceError: domain_marketing is not defined
console-api https://static.im-g.pl/wpc/frontend/portal-core-79/dist/desktop/ads/preload.74711296804972b7e04a.js 0:1483 "%cAdsSlots" "color: #fed049" "nodes found: 0"
console-api https://static.im-g.pl/wpc/frontend/portal-core-79/dist/desktop/ads/preload.74711296804972b7e04a.js 0:1741 "%cAdsSlots" "color: #fed049" "slots found: 0"
console-api https://static.im-g.pl/wpc/frontend/portal-core-79/dist/desktop/ads/preload.74711296804972b7e04a.js 0:1974 "%cAdsSlots" "color: #fed049" "all slots ready"
console-api https://static.im-g.pl/wpc/frontend/portal-core-79/dist/desktop/read-later/preload.7b4f037590f40055ba0e.js 0:12144 "%cReadLater" "color: #faa" "starting"
javascript https://bis.gazeta.pl/info/mapa2.js 4 A parser-blocking, cross site (i.e. different eTLD+1) script, https://gazeta.hit.gemius.pl/

In [25]:
log

{'level': 'INFO',
 'message': 'https://cdn.ampproject.org/rtv/012305252018000/amp4ads-v0.mjs 1:225973 "Powered by AMP ⚡ HTML – Version 2305252018000" "https://wyborcza.pl/0,177851.html"',
 'source': 'console-api',
 'timestamp': 1687499561403}

### Post / get requests

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time
import json

In [19]:
driver = webdriver.Chrome()
driver.get('https://wyborcza.pl/0,0.html')

start_time=time.time()
end_time=start_time+30

# Collect requests during the specified duration
while time.time() < end_time:
#     # Collect POST requests
#     post_requests = driver.execute_script("""
#         let performanceEntries = performance.getEntriesByType('navigation');
#         let entries = [];

#         for (let entry of performanceEntries) {
#             if (entry.initiatorType === 'xmlhttprequest' && entry.transferSize > 0) {
#                 entries.push(entry);
#             }
#         }

#         return entries;
#     """)
    # Collect POST requests
    post_requests = driver.execute_script("""
        let entries = performance.getEntriesByType('resource');
        let postRequests = [];

        for (let entry of entries) {
            if (entry.initiatorType === 'xmlhttprequest' && entry.transferSize > 0 && entry.nextHopProtocol === 'http/1.1') {
                if (entry.name.startsWith('http://') || entry.name.startsWith('https://')) {
                    postRequests.push(entry);
                }
            }
        }

        return postRequests;
    """)
    get_requests = driver.execute_script("""
        let performanceEntries = performance.getEntriesByType('resource');
        let entries = [];

        for (let entry of performanceEntries) {
            if (entry.initiatorType === 'xmlhttprequest' && entry.transferSize > 0) {
                entries.push(entry);
            }
        }

        return entries;
    """)   

    # Extract POST request details, payload, and headers
    for request in post_requests:
        request_url = request.get('name')
        request_method = 'POST'
        request_body_size =  request.get('transferSize')
        print(f"POST Request URL: {request_url}")
        print(f"Request Method: {request_method}")
        print(f"Request Body Size: {request_body_size} bytes")

        # Get the request payload
        request_payload = driver.execute_script(f"""
            let entries = performance.getEntriesByName('{request_url}');
            let payload = null;

            if (entries.length > 0) {{
                payload = entries[0].requestPayload;
            }}

            return payload;
        """)

        if request_payload:
            print(f"Request Payload: {request_payload}")

        # Get the request headers
        request_headers = driver.execute_script(f"""
            let entries = performance.getEntriesByName('{request_url}');
            let headers = null;

            if (entries.length > 0) {{
                headers = entries[0].requestHeaders;
            }}

            return headers;
        """)

        if request_headers:
            print("Request Headers:")
            for header_name, header_value in request_headers.items():
                print(f"{header_name}: {header_value}")

        print("----------")


#     # Extract GET request details
#     for request in get_requests:
#         request_url = request.get('name')
#         request_method = 'GET'
#         request_body_size = request.get('transferSize')
#         print(f"GET Request URL: {request_url}")
#         print(f"Request Method: {request_method}")
#         print(f"Request Body Size: {request_body_size} bytes")
#         print("----------")

    # Wait for a short interval before collecting requests again
    time.sleep(1)

# Close the Selenium webdriver
driver.quit()

In [20]:
import time
from selenium import webdriver

# Set up the Selenium webdriver
driver = webdriver.Chrome()  # Replace with the appropriate webdriver for your browser

# Perform necessary actions to interact with the web application
driver.get('https://wyborcza.pl/0,0.html')  # Replace with the URL of the web application

# Wait for the page to load and any network requests to occur
time.sleep(15)  # Adjust the duration as needed

# Get the network requests captured by the browser developer tools
network_requests = driver.execute_script("""
    var performanceEntries = window.performance.getEntries();
    var requests = [];

    for (var i = 0; i < performanceEntries.length; i++) {
        var entry = performanceEntries[i];
        if (entry.initiatorType === 'xmlhttprequest' && entry.transferSize > 0) {
            requests.push(entry);
        }
    }

    return requests;
""")

# Extract POST request details
for request in network_requests:
    request_url = request.get('name')
    request_method = 'POST'
    request_body_size =  request.get('transferSize')
    print(f"POST Request URL: {request_url}")
    print(f"Request Method: {request_method}")
    print(f"Request Body Size: {request_body_size} bytes")
    print("----------")

# Close the Selenium webdriver
driver.quit()

In [22]:
network_requests

[]

### Tak jak w postman

In [ ]:
import requests

# Step 1: Obtain an access token
auth_endpoint = 'http://keycloak.example.com/auth/realms/{realm}/protocol/openid-connect/token'

auth_data = {
    'grant_type': 'password',
    'client_id': 'your_client_id',
    'username': 'your_username',
    'password': 'your_password'
}

response = requests.post(auth_endpoint, data=auth_data)
response_data = response.json()
access_token = response_data['access_token']

# Step 2: Set up headers for subsequent requests
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Step 3: Navigate to the web page
web_page_url = 'http://example.com/protected-page'
response = requests.get(web_page_url, headers=headers)

# Step 4: Send a specific POST request
post_url = 'http://example.com/api/endpoint'
post_data = {
    'key1': 'value1',
    'key2': 'value2'
}

response = requests.post(post_url, json=post_data, headers=headers)

# Step 5: Process the response
print(response.status_code)
print(response.text)

In the above code:

Step 1 performs the authentication request to obtain an access token from the Keycloak server.

Step 2 sets up the headers for subsequent requests, including the access token for authentication.

Step 3 uses requests.get() to navigate to the web page that requires Keycloak authentication.

Step 4 sends a specific POST request to the desired endpoint.

Step 5 processes the response from the POST request.

Make sure to replace the placeholders {realm}, your_client_id, your_username, your_password, http://example.com/protected-page, and http://example.com/api/endpoint with the actual values corresponding to your Keycloak setup and the target web page and API endpoint.

You may need to customize the request parameters, headers, and body according to the requirements of the specific web page and API you are interacting with.

Yes, you can perform the navigation and POST request in headless mode using Python and the requests library. However, please note that the requests library alone cannot execute JavaScript or interact with dynamic elements on a web page. If the web page relies heavily on JavaScript or requires user interaction, you may need to consider using a browser automation tool like Selenium.

Here's an example of performing the navigation and POST request in headless mode using the requests library:

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Step 1: Set up Chrome options for headless browsing
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Step 2: Create a new Chrome driver instance
driver = webdriver.Chrome(options=chrome_options)

# Step 3: Obtain an access token
auth_endpoint = 'http://keycloak.example.com/auth/realms/{realm}/protocol/openid-connect/token'

auth_data = {
    'grant_type': 'password',
    'client_id': 'your_client_id',
    'username': 'your_username',
    'password': 'your_password'
}

response = requests.post(auth_endpoint, data=auth_data)
response_data = response.json()
access_token = response_data['access_token']

# Step 4: Set up headers for subsequent requests
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Step 5: Navigate to the web page
web_page_url = 'http://example.com/protected-page'
driver.get(web_page_url)

# Step 6: Send a specific POST request
post_url = 'http://example.com/api/endpoint'
post_data = {
    'key1': 'value1',
    'key2': 'value2'
}

response = requests.post(post_url, json=post_data, headers=headers)

# Step 7: Process the response
print(response.status_code)
print(response.text)

# Step 8: Close the Chrome driver
driver.quit()


In this example, we use Selenium WebDriver to launch a headless Chrome browser. The browser is then used to navigate to the web page. After the navigation is complete, we use the requests library to send the POST request.

Make sure to install the required packages by running pip install selenium before running the code.

Remember to replace the placeholders {realm}, your_client_id, your_username, your_password, http://example.com/protected-page, and http://example.com/api/endpoint with the actual values for your Keycloak setup and the target web page and API endpoint.

Note that using browser automation tools like Selenium allows for more dynamic interactions with web pages but may require additional setup and resources compared to simple requests with the requests library.

### z serverem proxy

In [1]:
import requests
import http.server
import socketserver
import urllib.parse
from keycloak import KeycloakOpenID
from keycloak.exceptions import KeycloakAuthenticationError

# Define the handler for the HTTP server
class RedirectHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        # Extract the authorization code from the query parameters
        query_params = urllib.parse.parse_qs(urllib.parse.urlparse(self.path).query)
        authorization_code = query_params.get('code', [None])[0]

        # Print the authorization code
        if authorization_code:
            print("Authorization code:", authorization_code)
            # Exchange the authorization code for an access token
            token = keycloak_openid.token('http://localhost:8080/callback', code=authorization_code, grant_type='authorization_code')
            access_token = token['access_token']
            print("Access token:", access_token)
        else:
            print("Authorization code not found.")

        # Send a response back to the browser
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        self.wfile.write(b'Authorization code received. You can close this window.')

# Set up the Keycloak client
keycloak_url = 'https://keycloak.example.com/auth/'
realm_name = 'your_realm'
client_id = 'your_client_id'
keycloak_openid = KeycloakOpenID(server_url=keycloak_url, client_id=client_id, realm_name=realm_name)

# Start the local HTTP server to listen for the redirect URI
with socketserver.TCPServer(('localhost', 8080), RedirectHandler) as httpd:
    print("Local HTTP server started at http://localhost:8080/")

    # Get the authorization URL
    auth_url = keycloak_openid.authorization_url(redirect_uri='http://localhost:8080/callback', pkce=True)

    # Open the authorization URL in a web browser
    import webbrowser
    webbrowser.open(auth_url)

    # Wait for the authorization code to be received
    httpd.handle_request()

ModuleNotFoundError: No module named 'keycloak'